In [1]:
%pip install surprise

     -------------------------------------- 772.0/772.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-win_amd64.whl size=1131265 sha256=b985d433a2192cf1c5ac6b3325e4dafbf749a9af7c1d2f941de2229ea8645c85
  Stored in directory: c:\users\rakla\appdata\local\pip\cache\wheels\af\db\86\2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [13]:
def readAppr(path):
    f = gzip.open(path, 'rt')
    for l in f:
        u,i,ts = l.strip().split(' ')
        yield u,i,ts

def readOwners(path):
    f = gzip.open(path, 'rt')
    for l in f:
        u,i = l.strip().split(' ')
        yield u,i


In [14]:
interactions = []
for l in readAppr("Behance_appreciate_1M.gz"):
    interactions.append(l)

creations = []
for l in readOwners("Behance_Item_to_Owners.gz"):
    creations.append(l)

## Exploratory Data Analysis

In [17]:
#size of dataset
len(creations)
len(interactions)

1000000

In [22]:
imgsperUser = defaultdict(list)
userperImgs  = defaultdict(list)
createdperUser = defaultdict(list)
usersperCreation = defaultdict(list)
for u,i,_ in interactions:
    imgsperUser[u].append(i)
    userperImgs[i].append(u)

for u,c in creations:
    createdperUser[u].append(c)
    usersperCreation[c].append(u)


In [23]:
x = len(imgsperUser) #no of users
y = len(userperImgs) #no of images
z = len(createdperUser) #no. of creators
r = len(usersperCreation) # no of creations
x,y,z,r

(63497, 178788, 178788, 51487)

In [24]:
images_with_gt_1 = [c for c,u in usersperCreation.items() if len(u)>1 ]

In [31]:
len(images_with_gt_1)/r

0.6051042010604618

In [38]:
print(images_with_gt_1[122])
usersperCreation[images_with_gt_1[122]]

1169556


['00045353', '00208333', '00211406', '00387493']

In [37]:
createdperUser['00045353']

['1169556']

In [39]:
imgCountAll = defaultdict(int)
totalliked = 0

for user,img,_ in interactions:
    imgCountAll[img] += 1
    totalliked+= 1

mostPopular = [(imgCountAll[x], x) for x in imgCountAll]
mostPopular.sort()
mostPopular.reverse()
print(totalliked)


1000000


In [42]:
return2 = set()
count = 0
threshold = 0.75
for imgCount, img in mostPopular:
    count += imgCount
    return2.add(img)
    if count > totalliked*threshold:
        break

In [43]:
def Jaccard(s1, s2):
  if not isinstance(s1,set):
    s1 = set(s1)
  if not isinstance(s2,set):
    s2 = set(s2)
  numer = len(s1.intersection(s2))
  denom = len(s1.union(s2))
  if denom == 0:
      return 0
  return numer / denom

def predictRead(user, img):
  Jsimilarities = []
  for d in imgsperUser[user]:
    #d is img user has liked in training
    if d == img: continue
    Jsimilarities.append(Jaccard(userperImgs[img],userperImgs[d]))
  mx = 0
  if Jsimilarities:
      mx = max(Jsimilarities)
  elif len(Jsimilarities) == 0:
    mx = 0
  if img in return2:
      return 1 + mx
  return mx

In [54]:
likedValid = interactions[990000:]
likedValid = [i + (1,) for i in likedValid]


imgsperUserVal = defaultdict(list)
for u,img,_,_ in likedValid:
    imgsperUserVal[u].append(img)

len(likedValid)

10000

In [55]:
all_books = set(userperImgs.keys())
val_pairs = set()
for user in imgsperUserVal.keys():
  no_books_read = len(imgsperUserVal[user])
  books_not_read = list(all_books.difference(set(imgsperUser[user])))
  for _ in range(no_books_read):
    book = random.choice(books_not_read)
    if (user,book, 'NA', 0) not in val_pairs:
      book = book
    else:
      while (user,book, 'NA', 0) in val_pairs:
        book = random.choice(books_not_read)
    val_pairs.add((user,book,'NA',0))
    likedValid.append((user,book,'NA',0)) 

len(likedValid)

20000

In [56]:
preds = [predictRead(u,b) for u,b,_,_ in likedValid]
correct = sum([likedValid[i][3]  == preds[i] for i in range(len(likedValid))])
y = [x for _,_,_,x in likedValid]
acc4 = correct/len(preds)
acc4 

0.3131